In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

- 거래 금액과 상관없이 앞으로 어떤 고객이 특정 거래를 할 것인지를 파악
- 

- 플랫폼 결정 : 코랩 
- 템플릿
    - 데이터와 분석 목적을 파악한 후 어떤 프로세스에 따라 분석을 진행할 건지(어떤 걸 할건지!)
    - 잘 정리된 EDA 같이 보면서 데이터 파악 

train
- ID_code(string)
- target
- 200 numerical variables, named from var_0 to var_199

In [ ]:
# 라이브러리 임포트

# data manipulation
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set()

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

# sklearn models & tools
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA

# ignore warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

import os
print(os.listdir("../input"))

In [ ]:
submission = pd.read_csv('/kaggle/input/santander-customer-transaction-prediction/sample_submission.csv')
train = pd.read_csv('/kaggle/input/santander-customer-transaction-prediction/train.csv')
test = pd.read_csv('/kaggle/input/santander-customer-transaction-prediction/test.csv')

In [ ]:
train.info()

In [ ]:
test.info()

### ID_code 빼고 전부 numerical variable

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.describe()

In [ ]:
train.isnull().sum().sum()

In [ ]:
test.isnull().sum().sum()

### 결측치 없음

In [ ]:
train.target.value_counts()

In [ ]:
sns.countplot(train.target)

In [ ]:
train.loc[train.target==1].shape[0] / train.loc[train.target==0].shape[0]

### class imbalanced 문제 있음 

- stratifiedKFold
- 성능평가 : ?

In [ ]:
# ID_code 변수 제거
train["Id"] = train.index.values
original_trainid = train.ID_code.values
train.drop("ID_code", axis=1, inplace=True)

test["Id"] = test.index.values
original_testid = test.ID_code.values
test.drop("ID_code", axis=1, inplace=True)

correlation 구해서 선형관계 아님을 확인하고, 선형 모델 말고 다른 모델로 돌려보기 : basemodel : lightGBM

- EDA 필요!! 데이터 파악 필요!!
- 그런데 EDA를 어떻게 해야 할 지??
- EDA를 좀 더 해서 어떻게 모델링할지 정해야 한다.

In [ ]:
# 상관계수 평균
(train.corr()).mean().mean()

0.005로 상관계수가 매우 낮다!

In [ ]:
(train.drop(["target"], axis=1).corr()).mean().mean()

In [ ]:
train_correlations = train.drop(["target"], axis=1).corr()
train_correlations = train_correlations.values.flatten()
train_correlations = train_correlations[train_correlations != 1]

test_correlations = test.corr()
test_correlations = test_correlations.values.flatten()
test_correlations = test_correlations[test_correlations != 1]

plt.figure(figsize=(20,5))
sns.distplot(train_correlations, color="Red", label="train")
sns.distplot(test_correlations, color="Green", label="test")
plt.xlabel("Correlation values found in train (except 1)")
plt.ylabel("Density")
plt.title("Are there correlations between features?"); 
plt.legend();

모든 변수가 선형 상관관계가 아니다. 

- top 10 feature를 뽑아서 사용하면 데이터의 본질을 이해하는데 도움이 되고, 새로운 변수를 생성하는데 어떠한 아이디어를 얻을 수 있을 것이다.
- 랜덤 포레스트를 이용해서 중요 변수를 선택 -> 선형 관계가 없기 때문에 nonlinear model을 사용하면 좋을 것이다.(변수의 중요도(importance), 상호작용(interaction)을 발견하는데 도움)
- 랜덤 포레스트 사용하면 어떤 방식으로 top 10 feature를 뽑는가? 기준이 무엇인가?

In [ ]:
parameters = {'min_samples_leaf': [20, 25]}
forest = RandomForestClassifier(max_depth=15, n_estimators=15)
grid = GridSearchCV(forest, parameters, cv=3, n_jobs=-1, verbose=2, scoring=make_scorer(roc_auc_score))

In [ ]:
grid

In [ ]:
# 모델 학습
grid.fit(train.drop(["target"], axis=1).values, train.target.values)

In [ ]:
grid.best_params_

In [ ]:
grid.best_score_

점수(roc_auc_score)가 그닥 좋지 않다.

### feature 뽑기!! : starting point!

- 랜덤 포레스트로 돌림
- 성능은 안 좋음
- 여기서 중요 변수 뽑음

### feature importance
- **트리 기반 모델**(randomforest, xgboost, lightgbm 등)에서 기본적으로 feature importance를 API 혹은 모델 내장 함수로 제공
- API(plot_importance)를 import하거나 **feature_importances_** 내장 함수를 이용해서 손쉽게 구현
- 하지만 이를 통해 구한 지표가 절대적인 것이 아님.
- feature importance를 고려하여 특성별로 A/B test 를 진행하며 feature selection을 하는 것이 좋다.

#### feature importance 동작 원리
- feature importance는 트리기반 모델에서 사용됨
- 중요도를 구분하는데에는 트리의 분할과 밀접한 관련이 있음.
- 즉 특정 변수가 트리를 분할하는데 얼마나 기여를 했는지에 따라 중요도가 결정됨

**트리 분할**
- 트리는 순수도를 기준으로 분할. 즉 가장 잘 분류시키는 변수를 순수도라는 기준으로 판단하는 것
- 순수도 계산 : 엔트로피, 지니계수
- 특정 변수로 인한 순수도의 변화량을 파악하기 위해(특정 변수의 중요도를 파악하기 위해 순수도의 변화량을 계산) <u>엔트로피를 활용한 정보이득량</u> 혹은 <u>지니계수를 이용한 지니 split</u>을 계산 -> 이를 기준으로 트리 분할
- 이때 '정보이득량이 가장 높은' 혹은 '지니 split이 가장 낮은' 변수를 선택하여 트리를 분할하고, 중요도에 반영된다.  

**feature importance는 왜 절대적인 지표가 되지 못할까?**
- feature importance는 노드가 분기 할때의 정보 이득 혹은 지니 계수만을 고려하여 중요도를 부여하기 때문에 과적합에 대해 고려하지 못함

In [ ]:
# 모델의 내장 함수인 feature_importances_
grid.best_estimator_.feature_importances_ # shape : 200

# 어레이 형태로 반환

In [ ]:
# np.argsort(importances)[::-1][0:n_top] 

# 인덱스 오름차순 정렬 -> 뒤집어서 내림차순 => 상위 5개의 인덱스

In [ ]:
# train.drop(["target", "ID_code"], axis=1).columns.values

In [ ]:
n_top = 5

In [ ]:
importances = grid.best_estimator_.feature_importances_ # 변수 중요도
idx = np.argsort(importances)[::-1][0:n_top] # 상위 5개의 인덱스
feature_names = train.drop(["target"], axis=1).columns.values # 변수 이름

# 변수 중요도 기준 상위 5개 변수의 중요도 시각화
plt.figure(figsize=(20,5))
sns.barplot(x=feature_names[idx], y=importances[idx]);
plt.title("What are the top important features to start with?");

변수를 뽑아서 얘네를 가지고 데이터 이해 여행을 떠나보자~

Q. 왜 랜덤 포레스트로 중요 변수를 뽑았는지?

Q. 변수 선택 방법으로는 또 어떤 것이 있는지?

Q. 중요 변수를 몇개를 뽑아서 탐색하는 것이 전체 EDA에 어떤 도움이 되는지? 

- train 데이터의 target에 대한 변수들의 분포가 어떤지?

- 선택된 상위 변수에 대해 train과 test 데이터의 형상의 불일치를 관찰할 수 있는지?

In [ ]:
train.var_81

In [ ]:
(train.loc[train.target==0, feature_names[idx][0]]).mean()  # var81

In [ ]:
fig, ax = plt.subplots(n_top,2,figsize=(20,5*n_top))

for n in range(n_top):
    sns.distplot(train.loc[train.target==0, feature_names[idx][n]], ax=ax[n,0], color="Orange", norm_hist=True)
    sns.distplot(train.loc[train.target==1, feature_names[idx][n]], ax=ax[n,0], color="Red", norm_hist=True)
    sns.distplot(test.loc[:, feature_names[idx][n]], ax=ax[n,1], color="Mediumseagreen", norm_hist=True)
    ax[n,0].set_title("Train {}".format(feature_names[idx][n]))
    ax[n,1].set_title("Test {}".format(feature_names[idx][n]))
    ax[n,0].set_xlabel("")
    ax[n,1].set_xlabel("")

- some peeks for variables 81, 12
- test 데이터에서 누적 밀도가 더 낮다

- train의 target=0과 test가 비슷한 모양이다.
- train의 target=1이 test는 많이 다르다. 왜 다를까?
- test에도 target=0이 훨씬 많을 것이다.(거래를 하지 않을 고객) => class imbalaced
- target=1은 데이터의 수가 적다.(거래를 할 고객)
- target=1과 target=0의 분포 모양이 달라야 확실하게 구분 가능하고, 이러한 변수를 사용해야 예측 정확도를 높일 수 있다. 

- 그러면 target에 따라 데이터의 분포가 많이 다른 변수들이 선택된 것일 거다.

In [ ]:
top = train.loc[:, feature_names[idx]]
top.describe()

#### 이항분포
- 질량함수 : 1, 2, 3 (x값 int)
- 밀도함수 : 연속된 값 (x값 float)

- 

In [ ]:
top.join(train.target)

In [ ]:
# scatter plot
# top = top.join(train.target)
# sns.pairplot(top, hue="target")

- target=1인 데이터들이 갑자기 누적되어서 거의 넘어가지 않는 날카로운 지점이 있다.
- 예를 들어, 81번 변수에서는 10에, 12번 변수에서는 13.5에 데이터가 누적되어 날카롭게 솟은 지점이 있다.(limit)
- This finding could be a nice entry point(진입점) for further feature engineering. => 이후 피처 엔지니어링에 좋은 출발선이 될 거다.?

- train, test PDF 동일!!
- train의 target=0과 target=1의 PDF 꽤 비슷함

In [ ]:
test.head()

In [ ]:
y_proba = grid.predict_proba(test.values)
y_proba_train = grid.predict_proba(train.drop('target', axis=1).values)

In [ ]:
(y_proba_train[:,1]).mean()

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(20,8))
sns.distplot(y_proba_train[train.target==1,1], norm_hist=True, color="mediumseagreen",
             ax=ax[0], label="1")
sns.distplot(y_proba_train[train.target==0,1], norm_hist=True, color="coral",
             ax=ax[0], label="0")
sns.distplot(y_proba[:,1], norm_hist=True,
             ax=ax[1], color="purple")
ax[1].set_xlabel("Predicted probability for test data");
ax[1].set_ylabel("Density");
ax[0].set_xlabel("Predicted probability for train data");
ax[0].set_ylabel("Density");
ax[0].legend();

- train 데이터에서 1이라고 예측한 것들의 확률값도 그닥 1에 가깝지 않다. -> cut-off가 중요할까? 했지만 평가 지표가 auc니까 상관없네.
    - 곡선이 완만하긴 하지만 제일 높은 값은 0.2 정도
- test는 대부분 0에 가까운 확률값으로 예측했다. -> 당연 : 0이 많으니까
- 몇 없는 1을 찾아내는 것이 중요!

- 이 예측값을 제출했더니 public leaderboard에서 점수 0.662 받음
- submission["target"] = y_proba

### feature engineering

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
original_features = train.drop(["target", "Id"], axis=1).columns.values
original_features

#### 반올림 및 분위수 기반 binning

- pd.qcut() : 수치형 변수를 특정 구간으로 나눈 범주형 레이블을 생성
    - 데이터를 동일한 길이로 나누는 것

In [ ]:
top

In [ ]:
pd.qcut(
        train.loc[:, 'var_81'].values,
        q=10,
        labels=False)

In [ ]:
np.round(train.loc[:, 'var_81'].values)

In [ ]:
# 중요변수 qcut한 변수 생성
encoder = LabelEncoder()
for your_feature in top.drop("target", axis=1).columns.values:
    train[your_feature + "_qbinned"] = pd.qcut(
        train.loc[:, your_feature].values,
        q=10,
        labels=False
    )
    train[your_feature + "_qbinned"] = encoder.fit_transform(
        train[your_feature + "_qbinned"].values.reshape(-1, 1)
    )

In [ ]:
# 중요 변수 반올림한 변수 생성
encoder = LabelEncoder()
for your_feature in top.drop("target", axis=1).columns.values:
    train[your_feature + "_rounded"] = np.round(train.loc[:, your_feature].values)
    train[your_feature + "_rounded_10"] = np.round(10*train.loc[:, your_feature].values)
    train[your_feature + "_rounded_100"] = np.round(100*train.loc[:, your_feature].values)

In [ ]:
# test에도 같이 적용

# 중요변수 qcut한 변수 생성
encoder = LabelEncoder()
for your_feature in top.drop("target", axis=1).columns.values:
    test[your_feature + "_qbinned"] = pd.qcut(
        test.loc[:, your_feature].values,
        q=10,
        labels=False
    )
    test[your_feature + "_qbinned"] = encoder.fit_transform(
        test[your_feature + "_qbinned"].values.reshape(-1, 1)
    )

# 중요 변수 반올림한 변수 생성
encoder = LabelEncoder()
for your_feature in top.drop("target", axis=1).columns.values:
    test[your_feature + "_rounded"] = np.round(test.loc[:, your_feature].values)
    test[your_feature + "_rounded_10"] = np.round(10*test.loc[:, your_feature].values)
    test[your_feature + "_rounded_100"] = np.round(100*test.loc[:, your_feature].values)

In [ ]:
train.head(10)

In [ ]:
# pd.qcut() : 데이터를 동일한 길이로 나눔
train['var_81_qbinned'].value_counts()

#### 새로운 변수 중요도

In [ ]:
# KFold로 데이터 나눠서 RandomForestClassifier 모델 돌림

cv = StratifiedKFold(n_splits=3, random_state=0)
forest = RandomForestClassifier(max_depth=15, n_estimators=15, min_samples_leaf=20,
                                n_jobs=-1)

scores = []
X = train.drop("target", axis=1).values
y = train.target.values

for train_idx, test_idx in cv.split(X, y):
    x_train = X[train_idx]
    x_test = X[test_idx]
    y_train = y[train_idx]
    y_test = y[test_idx]
    
    forest.fit(x_train, y_train)
    y_proba = forest.predict_proba(x_test)
    y_pred = np.zeros(y_proba.shape[0])
    y_pred[y_proba[:,1] >= 0.166] = 1
    
    score = roc_auc_score(y_test, y_pred)
    print(score)
    scores.append(score)

print(np.round(np.mean(scores),4))
print(np.round(np.std(scores), 4))

In [ ]:
# 중요 변수 뽑기 : feature_importances_

importances = forest.feature_importances_
feature_names = train.drop("target", axis=1).columns.values
idx = np.argsort(importances)[::-1][0:30]

plt.figure(figsize=(20,5))
sns.barplot(x=feature_names[idx], y=importances[idx]);
plt.xticks(rotation=90);

### Gaussian Mixture Clustering 

#### Gaussian Mixture Model(GMM) : 
- 개별 데이터가 가우시안 분포에 속한다고 가정을 한 상태에서 특정 정규분포에 속할 확률을 추정 => 확률 기반 군집화
- 가우시안 분포 = 정규분포
- 전체 데이터셋은 서로 다른 정규분포 형태를 가진 여러 가지 확률 분포 곡선으로 구성될 수 있으며, 이러한 서로 다른 정규분포에 기반해 군집화를 수행하는 것이 GMM 군집화 방식이다.
- 예를 들어 1000개의 데이터가 있다면 이를 구성하는 여러 개의 정규 분포 곡선을 추출하고, 개별 데이터가 이 중 어떤 정규분포에 속하는지 결정하는 방식
- 이와 같은 방식은 GMM에서는 모수 추정이라고 한다. 대표적으로 2가지 추정
    - 1) 개별 정규 분포의 평균과 분산
    - 2) 각 데이터가 어떤 정규 분포에 해당되는지의 확률
- GaussianMixture 객체의 주요 파라미터 : n_components
    - GaussianMixture의 모델의 총 개수. 군집화 개수
- GMM이 특히 잘 적용되는 데이터 분포 : 타원형으로 길게 늘어진 데이터 분포

#### RobustScaler
- 표준 정규화
- sklearn.preprocessing.RobustScaler
- 중앙값(median)과 IQR(interquartile range, 사분위값) 사용. 
- **이상치의 영향을 최소화**
- 중앙값을 제거하고 Quantile 범위(기본값은 IQR)에 따라 데이터를 스케일링
- IQR은 1분위(25분위)와 3분위(75분위) 사이의 범위이다.
- 데이터에서 이상치의 존재를 확인하고 그래서 이 스케일러를 사용하겠다고 하면 되겠다. 

In [ ]:
col1 = "var_81"
col2 = "var_12"
N=70000

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(20,10))
sns.kdeplot(train[col1].values[0:N], train[col2].values[0:N])
ax.scatter(train[col1].values[0:N], train[col2].values[0:N],
           s=2, c=train.target.values[0:N], cmap="coolwarm", alpha=0.5)
ax.set_xlabel(col1)
ax.set_xlabel(col2);

In [ ]:
# fig, ax = plt.subplots(1,1, figsize=(20,10))
sns.kdeplot(train[col1].values[0:N], train[col2].values[0:N])

In [ ]:
# var_81
sns.kdeplot(train[col1].values[0:N])

In [ ]:
# var_12
sns.kdeplot(train[col2].values[0:N])

In [ ]:
combined = train.drop(["target", "Id"], axis=1).append(test.drop("Id", axis=1))
combined.shape

In [ ]:
max_components = 10
start_components = 3
n_splits = 3
K = train.shape[0] # 400000

X = train.loc[:, original_features].values[0:K]
y = train.target.values[0:K]

In [ ]:
np.arange(start_components, max_components, 1)

In [ ]:
seeds = np.random.RandomState(0).randint(0,100, size=(max_components-start_components))
seeds

In [ ]:
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
fit_gaussians = False

In [ ]:
if fit_gaussians:
    components = np.arange(start_components, max_components, 1)
    kf = StratifiedKFold(random_state=0, n_splits=n_splits)
    
    scores = np.zeros(shape=(max_components-start_components, n_splits))

    for m in components:
        split=0
        print("Components " + str(m))
        for train_index, test_index in kf.split(X_scaled, y):
            print("Split " + str(split))
            x_train, x_test = X_scaled[train_index], X_scaled[test_index]
            gm = GaussianMixture(n_components=m, random_state=seeds[m-start_components])
            gm.fit(x_train)
            score = gm.score(x_test)
            scores[m-start_components,split] = score
            split +=1
    
    print(np.round(np.mean(scores, axis=1), 2))
    print(np.round(np.std(scores, axis=1), 2))
    best_idx = np.argmax(np.mean(scores, axis=1))
    best_component = components[best_idx]
    best_seed = seeds[best_idx]
    print("Best component found " + str(best_component))
    
else:
    best_seed = seeds[0]
    best_component = 3

In [ ]:
X = train.loc[:, original_features].values

gm = GaussianMixture(n_components=best_component, random_state=best_seed)
X_scaled = scaler.transform(X)
gm.fit(X_scaled) # 스케일한 데이터로 모델 학습

In [ ]:
X

In [ ]:
gm.score_samples(X_scaled) # 각 샘플에 대한 개별 점수 배열을 반환

In [ ]:
train["cluster"] = gm.predict(X_scaled)
train["logL"] = gm.score_samples(X_scaled)
test["cluster"] = gm.predict(test.loc[:, original_features].values)
test["logL"] = gm.score_samples(test.loc[:, original_features].values)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb

In [ ]:
features = [c for c in train.columns if c not in ['Id', 'target']]

cols = ["target","Id"]
X = train.drop(cols, axis=1)
y = train["target"]

X_test = test.drop("Id",axis=1)

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

#tree_model = DecisionTreeClassifier(random_state=0, max_depth=5, min_samples_split=5).fit(train_X, train_y)
# test = test.drop('Id', axis=1)

In [ ]:
X_test.shape

In [ ]:
train_X.shape

In [ ]:
val_X.shape

## 모델링!!

### lightGBM

- https://www.kaggle.com/mjbahmani/santander-ml-explainability

In [ ]:
# params is based on following kernel https://www.kaggle.com/brandenkmurray/nothing-works
# 모델의 정확도는 설정된 파라미터 값에 전적으로 달려 있다.
params = {'objective' : "binary",  
               'boost':"gbdt", # gbdt : Gradient Boosting Desicion Tree # 실행하고자 하는 알고리즘 타입 정의
               'metric':"auc",
               'boost_from_average':"false",
               'num_threads':8,
               'learning_rate' : 0.01, # 최종 결과에 대한 각각의 Tree에 영향을 미치는 변수
               'num_leaves' : 13, # 전체 Tree의 leave 수. Tree 모델의 복잡성을 컨트롤하는 주요 파라미터.
                # 이상적으로 num_leaves의 값은 2^(max_depth) 값보다 적거나 같아야 한다. 많으면 과적합 유발  
               'max_depth':-1,  # tree의 최대 깊이
               'tree_learner' : "serial",
               'feature_fraction' : 0.05, # 모델이 tree를 만들 때 매번 각각의 iteration에서 파라미터 중 5%를 랜덤하게 선택
               'bagging_freq' : 5,
               'bagging_fraction' : 0.4, # 매번 iteration을 돌 때 사용되는 데이터의 일부를 선택하는데 트레이닝 속도를 높이고 과적합을 방지할 때 주로 사용
               'min_data_in_leaf' : 80, # Leaf가 가지고 있는 최소한의 레코드 수, 과적합 해결에 사용, 디폴트 20(최적값)
               'min_sum_hessian_in_leaf' : 10.0,
               'verbosity' : 1}

### 파라미터 튜닝

더 빠른 속도를 위해 :

- bagging_fraction과 baggin_freq 을 설정하여 bagging 을 적용
- feature_fraction을 설정하여 feature sub-sampling을 하기
- 작은 max_bin 값을 사용
- save_binary 를 값을 통해 다가오는 학습에서 데이터 로딩 속도를 줄이기
- parallel learning 병렬 학습을 적용
- early_stopping_round : 만약 어떤 validation 데이터 중 하나의 지표가 지난 early_stopping_round 라운드에서 향상되지 않았다면 학습을 중단한다. 이는 지나친 iteration을 줄이는데 도움이 된다.


더 나은 정확도를 위해 :

- 큰 max_bin 값을 사용 (아마 속도는 느려질 수 있습니다)
- 작은 learning_rate 값을 큰 num_iterations 값과 함께 사용
- 큰 num_leaves 값을 사용 (아마 과적합을 유발할 수도 있습니다)
- 더 큰 트레이닝 데이터를 사용
- dart를 사용
- 범주형 feature를 사용


과적합을 해결하기 위해 :

- 작은 max_bin 값을 사용
- 작은 num_leaves 값을 사용
- min_data_in_leaf 와 min_sum_hessian_in_leaf 파라미터를 사용
- bagging_fraction 과 bagging_freq 을 사용하여 bagging 을 적용
- feature_fraction을 세팅하여 feature sub-sampling을 하기
- lambda_l1, lambda_l2 그리고 min_gain_to_split 파라미터를 이용해 regularization (정규화) 를 적용
- max_depth 를 설정해 Deep Tree 가 만들어지는 것을 방지

[출처]https://nurilee.com/2020/04/03/lightgbm-definition-parameter-tuning/

In [ ]:
import time

In [ ]:
# %%time
# y_pred_lgb = np.zeros(len(X_test))

# num_round = 1000000 # 일반적으로 100 이상

# fold_n=5
# folds = StratifiedKFold(n_splits=fold_n, shuffle=True, random_state=10)

# for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
#     print('Fold', fold_n, 'started at', time.ctime())
#     X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
#     y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
#     # train 데이터를 LightGBM에 맞는 데이터 세트 포맷으로 변환
#     train_data = lgb.Dataset(X_train, label=y_train)
#     valid_data = lgb.Dataset(X_valid, label=y_valid)
        
#     lgb_model = lgb.train(params, train_data, num_round,#change 20 to 2000
#                     valid_sets = [train_data, valid_data], verbose_eval=1000, early_stopping_rounds = 3500)##change 10 to 200
            
#     y_pred_lgb += lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration)/5 # folds.n_splits

In [ ]:
test2 = pd.read_csv('/kaggle/input/santander-customer-transaction-prediction/test.csv')

In [ ]:
# submission_lgb = pd.DataFrame({
#         "ID_code": test2["ID_code"],
#         "target": y_pred_lgb
#     })
# submission_lgb.to_csv('submission_lgb.csv', index=False)

## 제출 : 스코어 0.84381 

### catboost
- https://www.kaggle.com/mjbahmani/santander-ml-explainability

In [ ]:
# from catboost import CatBoostClassifier,Pool

# train_pool = Pool(train_X, train_y)
# cat_model = CatBoostClassifier(
#                                iterations=3000,# change 25 to 3000 to get best performance 
#                                learning_rate=0.03,
#                                objective="Logloss",
#                                eval_metric='AUC',
#                               )
# cat_model.fit(train_X,train_y,silent=True)
# y_pred_cat = cat_model.predict(X_test)

In [ ]:
# submission_cat = pd.DataFrame({
#         "ID_code": test2["ID_code"],
#         "target": y_pred_cat
#     })
# submission_cat.to_csv('submission_cat.csv', index=False)

## 제출 : 스코어 0.73194

### lightGBM
- https://www.kaggle.com/gpreda/santander-eda-and-prediction

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
target = train['target']

In [ ]:
folds = StratifiedKFold(n_splits=10, shuffle=False, random_state=44000)
oof = np.zeros(len(train))
predictions = np.zeros(len(test))
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
    print("Fold {}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx][features], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train.iloc[val_idx][features], label=target.iloc[val_idx])

    num_round = 1000000
    clf = lgb.train(params, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 3000)
    oof[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))

In [ ]:
roc_auc_score(target, oof)

In [ ]:
submission_lgb2 = pd.DataFrame({
        "ID_code": test2["ID_code"],
        "target": predictions
    })
submission_lgb2.to_csv('submission_lgb2.csv', index=False)

## 제출 : 스코어 0.89692

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,5))
sns.countplot(train.cluster, palette="Set2", ax=ax[0])
sns.distplot(train.logL, color="Dodgerblue", ax=ax[1]);

- Gaussian Mixture Model을 학습시킴으로써 로그 우도(log likelihood)를 최대화하고 있다.(logL)
- 로그 우도(logL)가 높을 수록 데이터가 가우시안 분포에 더 잘 맞는다는 것이다.
- 올바른 n_components를 선택하는 것이 어렵기 때문에 train 데이터의 stratified k fold를 사용하자.
- 이렇게 하면 train subset에 가우시안을 적합시키고, test subset에서 로그 우도가 얼마나 큰지 검정할 수 있다.
- 선택한 n_components에 대해 3번(k=3) 이 작업을 수행하면 solution의 안전성에 대한 정보를 얻을 수 있다.
- **n_components가 클수록 로그 우도 값이 감소하므로 n_components=3일 때 충분하다는 것을 알 수 있다.**

- **데이터 spot당 개별 점수(logL)는 밀도를 측정하는 척도로 이해할 수 있다.** 
- logL이 낮으면 데이터 spot이 다른 데이터 지점과 멀리 떨어진 곳에 있는 것이다. -> 멀리 떨어져 있다? -> 이상치인가?
- logL이 높으면 이웃이 많을 것이다.(주변에 데이터가 많다.) -> 이상치 아님
- **따라서 개별 logL-score는 데이터의 이상치에 대한 정보를 제공할 수 있다.**
    - 구체적으로 logL-score를 보고 어떻게 이상치를 확인하는데?
    - 가장 낮은 데이터들을 제거해야 되는 거야?

In [ ]:
train["logL"].sort_values()

In [ ]:
train["logL"].argsort() # 오름차순 -> 제일 큰 값의 인덱스가 맨 아래

In [ ]:
train.groupby("cluster").target.value_counts() / train.groupby("cluster").size() * 100

In [ ]:
# cluster_occupation
(train.groupby("cluster").target.value_counts() / train.groupby("cluster").size() * 100).loc[:, 1]

In [ ]:
# target_occupation
(train.groupby("target").cluster.value_counts() / train.groupby("target").size() * 100).loc[1, :]

In [ ]:
# 각 cluster가 target=1일 확률
cluster_occupation = train.groupby("cluster").target.value_counts() / train.groupby("cluster").size() * 100
cluster_occupation = cluster_occupation.loc[:, 1] 

# target=1인 데이터가 각 클러스터에 속할 확률
target_occupation = train.groupby("target").cluster.value_counts() / train.groupby("target").size() * 100
target_occupation = target_occupation.loc[1, :]
target_occupation.index = target_occupation.index.droplevel("target")

fig, ax = plt.subplots(1,2,figsize=(20,5))
ax[0].set_title("How many % of the data per cluster has hot targets?")
sns.barplot(cluster_occupation.index, cluster_occupation.values, ax=ax[0], color="cornflowerblue")
ax[0].set_ylabel("% of cluster data")
ax[0].set_ylim([0,100])

ax[1].set_title("How many % of total hot targets are in one cluster?")
sns.barplot(target_occupation.index, target_occupation.values, ax=ax[1], color="tomato")
ax[1].set_ylabel("% of hot targets")
ax[1].set_ylim([0,100]);

- cluster 2 has more hot target(1) than others
- hot target(1)의 대부분이 cluster 2에 위치한다.

- 클러스터링으로 데이터 분류
- 어떤 기준으로 클러스터링? -> 가우시안 분포도?
- 분류별로 target=1이 속한 비율 확인하기(그래프)
- 어떤 클러스터에 target=1인 데이터가 많은지 확인 가능, 순서도


In [ ]:
# # KFold로 데이터 나눠서 RandomForestClassifier 모델 돌림

# cv = StratifiedKFold(n_splits=3, random_state=0)
# forest = RandomForestClassifier(max_depth=15, n_estimators=15, min_samples_leaf=20,
#                                 n_jobs=-1)

# scores = []
# X = train.drop("target", axis=1).values
# y = train.target.values

# for train_idx, test_idx in cv.split(X, y):
#     x_train = X[train_idx]
#     x_test = X[test_idx]
#     y_train = y[train_idx]
#     y_test = y[test_idx]
    
#     forest.fit(x_train, y_train)
#     y_proba = forest.predict_proba(x_test)
#     y_pred = np.zeros(y_proba.shape[0])
#     y_pred[y_proba[:,1] >= 0.166] = 1
    
#     score = roc_auc_score(y_test, y_pred)
#     print(score)
#     scores.append(score)

# print(np.round(np.mean(scores),4))
# print(np.round(np.std(scores), 4))

In [ ]:
gm.means_[0]

In [ ]:
gm.means_.shape

In [ ]:
plt.figure(figsize=(20,5))
for n in range(gm.means_.shape[0]): # 클러스터 3개 -> gm.means_.shape[0] : 3
    plt.plot(gm.means_[n,:], 'o')
plt.title("How do the gaussian means look like?")
plt.ylabel("Cluster mean value")
plt.xlabel("Feature")

Only some features are important to separate the structure of the data.

- 그래서 어떤 피처를 사용해야 되는데?
- 이상치 처리는 어떻게 하는데?
- 피처 선택을 하긴 하는 거야?